<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/binance_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binance用データ蓄積クラスを使ったサンプル

`BinanceMarket`は、通過ペアー毎に以下の機能を提供します。

* 蓄積用データベースの生成（Sqlite)
* 約定データのダウンロードと更新
  * 過去データのダウンロード(Binance Webサイト)
  * リアルタイム更新(WebSocket)
* 約定データのDBからの取得と足の作成
  * 任意の期間の足の作成(`ohlcv`メソッド)
  * VAP(Value At Price)の計算(`vap`メソッド)
* オーダーの発行・キャンセル（REST API)　KEYとSECRETの設定が必要です。

また通過ペアーの設定は`BinanceConfig`クラスに定義してあります。パラーメータを修正することで任意の通過ペアーに対応可能ですが、テスト完了して提供しているものは以下になります。
* `BinanceConfig.BTCUSDT` 本番用BTCUSDTペアー（現物）
* `BinanceConfig.TEST_BTCUSDT`　テストネット用BTCUSDTペアー（現物）

また注意点としてはDBに書き込めるのは１プロセスのみとなります。複数プロセスが立ち上がっているとエラーになったりロックがかかったりします。

## 本サンプルの機能

本サンプルでは以下を行います。フレームワークがかなりの部分やってくれるのでシンプルに実行できることがわかると思います。

* BTCUSDTペアー現物を扱う
* 過去データのダウンロード
* OHLCVの作成とPlotlyによる描画
* VAP(Value At Price)の作成と描画
* WebSocketによるリアルタイムデータの受信（板情報）

## 必要ライブラリのインストール

`rbot`がBotFrameWorkの本体です。必要なライブラリを合わせてインストールします。２回目は実行不要です。

In [ ]:
# 必要ライブラリのインストール
! pip install --upgrade pip

# rbotがbot frameworkになります。PyPiに登録されているので、pipでインストールできます。
# まだ開発初期段階なので、インターフェースが変わる可能性があります。そのためバージョン指定しています。
! pip install --upgrade rbot==0.2.6

In [ ]:
# 必要ライブラリのインストール。環境によっては要・不要があるので適宜修正してください。
! pip install json2html
! pip install plotly
! pip install nbformat
! pip install pandas
! pip install polars
! pip install pyarrow
! pip install --upgrade ipywidgets
! pip install --upgrade ipympl

## Python コード本体

### 必要ライブラリのインポート

In [ ]:
import rbot
# BinanceMarketクラスは、BinanceのAPIを利用して、取引所の情報を取得するクラス
from rbot import BinanceMarket

# BinanceConfigクラスは、BinanceMarketクラスのコンストラクタに渡す設定クラス
from rbot import BinanceConfig

from rbot import init_log, init_debug_log
#init_debug_log()

print("rbot version =", rbot.__version__)

In [ ]:
# 必要ライブラリーのインポート
from json2html import *
from IPython.display import HTML

### BinanceMarketオブジェクトの生成

引数にBinanceConfigを設定してオブジェクトを生成します。

In [ ]:
# BinanceMarketクラスのインスタンスを生成(BTCUSDT現物取引の設定)
config = BinanceConfig.BTCUSDT

# json2htmlを利用して、設定情報を表示
HTML(json2html.convert(config.__str__()))

In [ ]:
# BinanceMarketクラスのインスタンスを生成(BTCUSDT現物取引の設定)
market = BinanceMarket(config)

# BinanceMarketのインスタンスを表示すると格納されているデータの情報が表示されます。
market

In [ ]:
#init_log()

In [ ]:
# 過去１日分のデータをダウンロード。ダウンロードしたデータは、marketオブジェクトの属性に格納される。
# Trueを指定すると、再ダウンロードを行う。Falseの場合はローカルに保存されている場合は、再ダウンロードを行わない。
# 比較的時間がかかる処理です。終わるまで少々お待ちください。完了すると取り込まれたレコード数が表示されます。
market.download(1, False)

### OHLCVの計算

BinanceMarketオブジェクトのohlcvメソッドを使うと任意の時間足でローソク足をつくることができます。戻り値はPolarsのDataFrame型です。

#### `ohlcv`メソッド

```
BinanceMarket#ohlcv(
    start_time=0, # 開始時刻(UNIX時間[us]) 0の場合はDBにある最初のレコードから
    end_time=0, # 終了時刻(UNIX時間[us])　 0の場合はDBにある最後のレコードまで
    window_sec=60 #OHLCV足の時間幅(秒)
)
```


In [ ]:
# データベースすべての期間,１分足でOHLCVを計算する。
ohlcv = market.ohlcv(start_time=0, end_time=0, window_sec=60)

In [ ]:
# ohlcvはpolarsのDataFrameオブジェクトとして格納されている。
ohlcv.head()

### (参考)Pandasへの変換

Polarsの情報はまだ不足気味。また他のライブラリとの互換性の問題でPandasを使いたい場合は、Polarsのデータフレームオブジェクトの`to_pandas`メソッドで簡単にPandasへ変換することができます。このとき`use_pyarrow_extension_array`を`True`に指定するとデータのコピーが発生せず高速です。

In [ ]:
# polarsからpandasへ変換する。
pd_ohlcv = ohlcv.to_pandas(use_pyarrow_extension_array=True)

In [ ]:
pd_ohlcv.head()

## Plotlyでローソク足を表示する。

Plotlyを使うと簡単にローソク足を表示できます。polarsのデータそのままで表示可能です。

In [ ]:
# OHLCVの表示
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_width=[0.2, 0.7])

fig.add_trace(
        go.Candlestick(
            x=ohlcv['time_stamp'],
            open=ohlcv['open'],
            high=ohlcv['high'],
            low=ohlcv['low'],
            close=ohlcv['close'],

        ),
        row=1, col=1
)

fig.add_trace(
        go.Bar(
            x=ohlcv['time_stamp'],
            y=ohlcv['volume'],
        ),
        row=2, col=1
)

#fig.update_layout(layout_xaxis_rangeslider_visible=False)

fig


### VAP(Volume At Price)の計算

`BinanceMarket#vap`で指定した期間のVAPを計算します。

```
BinanceMarket#vap(
    start_time = 0, # start_time: 開始時刻(UNIX時間[us]) 0の場合はDBにある最初のレコードから
    end_time = 0, # end_time:終了時刻(UNIX時間[us]) 0の場合はDBの最終レコードまで
    price_unit = 1 #価格をまとめる単位
)
```

In [ ]:
vap = market.vap(start_time=0, end_time=0, price_unit=1)

In [ ]:
vap

In [ ]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=vap['sell_volume'],
            y=vap['price'],
            fill='tozerox',
            name='sell'
        ),
        go.Scatter(
            x=vap['sell_volume'] + vap['buy_volume'],
            y=vap['price'],
            fill='tonextx',
            name='buy'
        ),
    ],
    layout=go.Layout(barmode='stack')
)

fig

## WebSocketによるリアルタイム情報取得
(注：ColabではWebSocketが動きません。ローカル環境で試してください）

なお、同時にTick（約定）情報もDBへリアルタイムに更新されていきます。
そのため２回目にOHLCV取得すると、リアルタイム分とバッチダウンロード分の間にギャップができます。
ここを埋めるAPIもあるのですが、また別途。

In [ ]:
# WebSocketデータの受信開始
market.start_market_stream()

from time import sleep
sleep(1)

In [ ]:

from time import sleep
import plotly.graph_objects as go

def trim(bids, asks):
    ask_spred =  asks['price'][-1] - asks['price'][0]
    bids_spred = bids['price'][0] - bids['price'][-1]

    if ask_spred < bids_spred:
        edge = bids['price'][0]
        bids = bids.filter(bids['price'] > edge - ask_spred * 3)
    else:
        edge = asks['price'][0]
        asks = asks.filter(asks['price'] < edge + bids_spred * 3)

    return bids, asks

fig = go.FigureWidget()

bids, asks = market.board
bids, asks = trim(bids, asks)

fig.add_scatter(
    x=bids['price'],
    y=bids['sum'],
    name='bids',
    marker_color='blue'
)

fig.add_scatter(
    x=asks['price'],
    y=asks['sum'],
    name='asks',
    marker_color='red'
)

fig.layout.title = 'Binance BTCUSDT Order Book'
fig

In [ ]:
# 0.1秒ごとに板情報を更新する（１００回ループ）

for i in range(100):
    bids, asks = market.board
    bids, asks = trim(bids, asks)
    bids_edge = bids['price'][0]
    asks_edge = asks['price'][0]

    fig.layout.title = f'Binance BTCUSDT Order Book (bids_edge={bids_edge}, asks_edge={asks_edge})'

    fig.data[0].x = bids['price']
    fig.data[0].y = bids['cusum']
    fig.data[1].x = asks['price']
    fig.data[1].y = asks['cusum']
    sleep(0.1)






# 以後テスト用

In [ ]:
# VAP内の縦横計算の確認

vap_sum = vap.sum()

print(vap_sum)

vap_sell_sum = vap_sum['sell_volume'][0]
vap_buy_sum = vap_sum['buy_volume'][0]
vap_volume_sum = vap_sum['volume'][0]

print('vap_sell_volume: ', vap_sell_sum)
print('vap_buy_volume: ', vap_buy_sum)
print('vap_volume: ', vap_volume_sum)

if vap_sell_sum + vap_buy_sum != vap_volume_sum:
    print('error')
else:
    print('OK')


In [ ]:
#　合計値がOHLCVとVAPで一致することを確認する。
ohlcvv = market.ohlcvv(start_time=0, end_time=0, window_sec=60)

sum = ohlcvv['volume'].sum()

sum1 = ohlcv['volume'].sum()
sum2 = vap['buy_volume'].sum() + vap['sell_volume'].sum()

print(f'OHLCVVの合計値: {sum}')
print(f'OHLCVの合計値: {sum1}')
print(f'VAPの合計値: {sum2}')
